<a href="https://colab.research.google.com/github/carlibeisel/mason_water_budget/blob/main/reach_precip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

By Carli Beisel

Created August, 2024

Purpose: Calculates precipitation for each Reach within Mason Drainage from Daymet.

## 1. Install packages and connect to Google Drive

In [1]:
# -----------------------------#
#   Installs Geemap Package
# -----------------------------#

import subprocess
!pip install geemap

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project = 'extract-gridmet')

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [2]:
# ----------------------------------------#
#   Import Libraries & Connect to Drive   #
# ----------------------------------------#


!pip install geopandas
import geopandas as gpd #import independent shapefile
import json #for metadata of shapefile
import os #for file paths
import numpy as np #for stats and arrays
import pandas as pd #for dataframes
!pip install pycrs
import pycrs

#Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

  Preparing metadata (setup.py) ... done
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32686 sha256=299f37756fdce51ff52916be8e54a99fbbf42de5b39ea1ade2fa7e05c8b0cc0e
  Stored in directory: /root/.cache/pip/wheels/47/1d/70/7a5bdf33347e7c75e95b06b1fa38f076a59a9506653cc24aff
Successfully built pycrs
Mounted at /content/drive


## 2. Import reach shapefiles and start/end dates to clip to dataset ##

In [39]:
## ------------------------------------------------------- ##
##   Import shapefile and start/end dates to clip dataset  ##
## ------------------------------------------------------- ##

# change start and end date here
start_date = '2024-3-1' # format is yyyy-mm-dd --> but exclude 0 from beg of days/months
end_date = '2024-7-1'

# create a df with all dates between start_date and end_date
date_range = pd.date_range(start=start_date, end=end_date)
dates_range = pd.DataFrame(date_range, columns=['Date'])

# import reach shapefiles for Mason Creek
shp_file = '/content/drive/MyDrive/Data/Mason/data_input/reach_shapefile/Mason Reaches/Purdum Reach/layers/purdum_reach.shp'
reach = emap.shp_to_ee(shp_file) # converts shapefile to feature in GEE

map = emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(reach, 0, 2), {}, 'POU')
map.addLayerControl()
map

Map(center=[43.615, -116.2023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [41]:
## ------------------------------------ ##
##        REACH PRECIPITATION           ##
## ------------------------------------ ##

# sums daily precipitation for each reach

precip = []

for i in range(len(dates_range)):
    daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate(start_date, end_date)
    daily_sum = daymet.select('prcp').map(lambda image: image.clip(reach).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=reach,
        scale=1000,
        bestEffort=True
    ).set('system:time_start', image.get('system:time_start')))

    precip.append(daily_sum)

pr = ee.ImageCollection.fromImages(precip)

In [52]:
# similar to above, but goes through one day at a time.
dates_list = dates_range['Date'].dt.strftime('%Y-%m-%d').tolist()  # Convert dates to a list of strings

precip = []

for i in dates_list:
    date = ee.Date(i)
    daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate(date, date.advance(1, 'day'))
    daily_sum = daymet.select('prcp').map(lambda image: image.clip(reach).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=reach,
        scale=1000,
        bestEffort=True
    ).set('system:time_start', image.get('system:time_start')))
    precip.append(daily_sum)

pr = ee.ImageCollection.fromImages(precip)

In [53]:
## ------------------------ ##
##   CALCULATE ZONAL STATS  ##
## ------------------------ ##

out_stats = os.path.join('/content/drive/MyDrive/Data/Mason/data_output/precip_out/precip_purdum.csv')
emap.zonal_statistics(pr, reach, out_stats, statistics_type='SUM', scale=1000)

Computing statistics ...


Exception: Collection.map: A mapped algorithm must return a Feature or Image.

In [ ]:
## ---------------------------------------------- ##
##   CREATE CLIMATE STAT FOR EACH POU AND EXPORT  ##
## ---------------------------------------------- ##

# delete excess and make for each reach

date = np.arange(dates_range)
precip = pd.read_csv('.csv')

names = reach['Name']

for i in range(len(names)):
  df = pd.DataFrame(date, columns=['Date'])
  df['NAME'] = names[i]
  df['prcp'] = precip.iloc[i,0:34].values
  out_path = os.path.join('.csv')
  df.to_csv(out_path)